In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd
import string
import re
import nltk
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
from keras.callbacks import ModelCheckpoint

#!pip install mxnet-cu80
#import mxnet as mx

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:
df = pd.read_csv('/content/gdrive/My Drive/Consumer_Complaints.csv')

In [0]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,08/09/2015,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,Company chooses not to provide a public response,Experian Information Solutions Inc.,NJ,08872,NaN,Consent not provided,Web,08/09/2015,Closed with non-monetary relief,Yes,No,1509954
1,01/29/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,NaN,NaN,"EQUIFAX, INC.",NY,10801,NaN,Consent not provided,Web,01/30/2019,Closed with non-monetary relief,Yes,NaN,3136759
2,10/13/2019,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,NaN,NaN,SANTANDER CONSUMER USA HOLDINGS INC.,GA,300XX,Servicemember,NaN,Web,10/13/2019,Closed with explanation,Yes,NaN,3404213
3,08/19/2015,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,Company chooses not to provide a public response,WELLS FARGO & COMPANY,CA,94526,NaN,Consent not provided,Web,08/19/2015,Closed with explanation,Yes,No,1527601
4,03/04/2016,Credit card,NaN,Billing disputes,NaN,I am dissatisfied with the current outcome of ...,NaN,DISCOVER BANK,NV,891XX,NaN,Consent provided,Web,03/04/2016,Closed with explanation,Yes,Yes,1816726


In [0]:
df = df.loc[df['Consumer complaint narrative'].isnull()==False,:]

In [0]:
df['issue_combined'] = df['Sub-issue']
df.loc[df['issue_combined'].isnull()==True,'issue_combined'] = df['Issue']

In [0]:
df['complaints'] = df['Consumer complaint narrative']

In [0]:
sample = df.sample(n=10000)

In [0]:
complaints = sample['complaints']
issues = sample['issue_combined']

In [0]:
complaints = complaints.apply(lambda x: str(x).lower())
issues = issues.apply(lambda x: str(x).lower())
issues = issues.apply(lambda x : 'SOS_ '+ x + ' _EOS')

In [0]:
def cleanup(text):
  cleaned = text
  cleaned = [re.sub("'", ' ', x) for x in cleaned]
  cleaned = [re.sub('"', ' ', x) for x in cleaned]
  cleaned = [re.sub(",", ' ', x) for x in cleaned]
  cleaned = [re.sub("xx", ' ', x) for x in cleaned]
  cleaned = [re.sub(" x ", ' ', x) for x in cleaned]
  exclude = set(string.punctuation)
  cleaned = [''.join(char for char in x if char not in exclude) for x in cleaned]
  numbers = set(string.digits)
  cleaned = [''.join(char for char in x if char not in numbers) for x in cleaned]
  cleaned = [re.sub("  ", ' ', x) for x in cleaned]
  return cleaned


In [0]:
complaints = cleanup(complaints)
issues = cleanup(issues)

In [0]:
stemmer = PorterStemmer()
#tokenizer = word_tokenize()
stop_words = set(stopwords.words('english')) 

In [0]:
words_complaints = set()
for complaint in complaints:
  for word in complaint.split():
    #word = stemmer.stem(word)
    if word not in words_complaints: #and word not in stop_words:
      words_complaints.add(word)

In [0]:
words_issues = set()
for issue in issues:
  for word in issue.split():
    #word = stemmer.stem(word)
    if word not in words_issues: #and word not in stop_words
      words_issues.add(word)

In [0]:
num_complaint_words = len(words_complaints)
num_issue_words = len(words_issues)

In [0]:
print(num_complaint_words)
print(num_issue_words)

20887
467


In [0]:
max_complaint_words = max([len(complaint.split()) for complaint in complaints])
max_issue_words = max([len(issue.split()) for issue in issues])

In [0]:
print(max_complaint_words)
print(max_issue_words)

5085
19


In [0]:
num_complaints = len(complaints)
num_issues = len(issues)

print(num_complaints)
print(num_issues)

10000
10000


In [0]:
encoder_word_to_int = dict()
encoder_int_to_word = dict()

decoder_word_to_int = dict()
decoder_int_to_word = dict()


In [0]:
input_words = list(words_complaints)
target_words = list(words_issues)

In [0]:
for row, word in enumerate(input_words):
  encoder_word_to_int[word] = row
  encoder_int_to_word[row] = word

In [0]:
for row, word in enumerate(target_words):
  decoder_word_to_int[word] = row
  decoder_int_to_word[row] = word

In [0]:
# initiate numpy arrays to hold the data that our seq2seq model will use:
encoder_input_data = np.zeros(
    (num_complaints, max_complaint_words),
    dtype='float32')
decoder_input_data = np.zeros(
    (num_issues, max_issue_words),
    dtype='float32')
decoder_target_data = np.zeros(
    (num_issues, max_issue_words, num_issue_words),
    dtype='float32')

In [0]:
# Process samples, to get input, output, target data:
for i, (input_text, target_text) in enumerate(zip(complaints, issues)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = encoder_word_to_int[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = decoder_word_to_int[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, decoder_word_to_int[word]] = 1.

In [0]:
vec_len       = 100   # Length of the vector that we will get from the embedding layer
latent_dim    = 200  # Hidden layers dimension 
dropout_rate  = 0.2   # Rate of the dropout layers
batch_size    = 40    # Batch size
epochs        = 3    # Number of epochs


In [0]:
# Define an input sequence and process it.
# Input layer of the encoder :
encoder_input = Input(shape=(None,))

# Hidden layers of the encoder :
encoder_embedding = Embedding(input_dim = num_complaint_words, output_dim = vec_len)(encoder_input)
encoder_dropout   = (TimeDistributed(Dropout(rate = dropout_rate)))(encoder_embedding)
encoder_LSTM      = LSTM(latent_dim, return_sequences=True)(encoder_dropout)
#CuDNNLSTM

# Output layer of the encoder :
encoder_LSTM2_layer = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM2_layer(encoder_LSTM)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# Set up the decoder, using `encoder_states` as initial state.
# Input layer of the decoder :
decoder_input = Input(shape=(None,))

# Hidden layers of the decoder :
decoder_embedding_layer = Embedding(input_dim = num_issue_words, output_dim = vec_len)
decoder_embedding = decoder_embedding_layer(decoder_input)

decoder_dropout_layer = (TimeDistributed(Dropout(rate = dropout_rate)))
decoder_dropout = decoder_dropout_layer(decoder_embedding)

decoder_LSTM_layer = LSTM(latent_dim, return_sequences=True)
decoder_LSTM = decoder_LSTM_layer(decoder_dropout, initial_state = encoder_states)

decoder_LSTM_2_layer = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_LSTM_2,_,_ = decoder_LSTM_2_layer(decoder_LSTM)

# Output layer of the decoder :
decoder_dense = Dense(num_issue_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_LSTM_2)

In [0]:

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_input, decoder_input], decoder_outputs)

model.summary()

# Define a checkpoint callback :
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    2088700     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 100)    0           embedding_1[0][0]                
____________________________________________________________________________________________

In [0]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
tf.compat.v1.keras.backend.set_session(tf.Session(config=config))

In [0]:
num_train_samples = 500
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data[:num_train_samples,:],
               decoder_input_data[:num_train_samples,:]],
               decoder_target_data[:num_train_samples,:,:],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.08 ) #,
          #callbacks = None) #callbacks_list)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 460 samples, validate on 40 samples
Epoch 1/3




460/460 [==============================] - 184s 401ms/step - loss: 1.7806 - val_loss: 1.5819
Epoch 2/3
460/460 [==============================] - 175s 380ms/step - loss: 1.5613 - val_loss: 1.5106
Epoch 3/3
460/460 [==============================] - 173s 375ms/step - loss: 1.4989 - val_loss: 1.4644


In [0]:
encoder_model = Model(encoder_input, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_LSTM(decoder_input #decoder_LSTM
    , initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_input] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

TypeError: ignored